In [111]:
import pandas as pd
import torch
import numpy as np
df = pd.read_csv('/Users/gclyne/thesis/data/cesm_data.csv')
arr = np.empty((0,30,10), int)
df.loc[(df['longitude'] == -81.25) & (df['latitude'] == 42.879581451416016)][0:30]
for lon in df['longitude'].unique():
    for lat in df['latitude'].unique():
        data_row = df.loc[(df['longitude'] == lon) & (df['latitude'] == lat)][0:30][['pr','tas','# lai','treeFrac','baresoilFrac','ps','cSoil','cCwd','cVeg','cLitter']].to_numpy()
        # print(data_row)
        arr = np.append(arr,data_row)
    


# df.loc[(df['longitude'] == -81.25) & (df['latitude'] == 42.879581451416016)][0:30][['pr','tas','# lai','treeFrac','baresoilFrac','ps']].to_numpy().shape


[[3.18043276e-05 2.80561157e+02 1.49282992e+00 3.45470276e+01
  1.60918331e+01 9.86259375e+04 9.06196404e+00 1.23027182e+00
  4.17318058e+00 2.68238515e-01]
 [3.31580814e-05 2.81467865e+02 1.45780408e+00 3.40852280e+01
  1.60790424e+01 9.88061250e+04 9.05838871e+00 1.23797691e+00
  4.11083317e+00 2.75562108e-01]
 [3.67320354e-05 2.79864502e+02 1.35064757e+00 3.36234207e+01
  1.60662518e+01 9.87749766e+04 9.05480099e+00 1.24473119e+00
  4.05237341e+00 2.78131455e-01]
 [3.46612360e-05 2.81124969e+02 1.40018308e+00 3.31616249e+01
  1.60534611e+01 9.88879453e+04 9.04967403e+00 1.24867821e+00
  4.00974941e+00 2.80183673e-01]
 [2.96442231e-05 2.80348450e+02 1.35222065e+00 3.26998291e+01
  1.60406666e+01 9.88552188e+04 9.05031681e+00 1.25824320e+00
  3.94899940e+00 2.91834533e-01]
 [3.55930642e-05 2.80359650e+02 1.41425085e+00 3.22380295e+01
  1.60278759e+01 9.87001953e+04 9.04998207e+00 1.26276064e+00
  3.90253711e+00 2.97963381e-01]
 [3.49372021e-05 2.80363983e+02 1.40020609e+00 3.17762356e

In [113]:
train_data = arr.reshape(513,30,10)

In [ ]:
from typing import Tuple
def get_batch(source: torch.Tensor, i: int) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size]
    """
    seq_len = 30
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target


In [114]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torch as T
import math

class TransformerModel(T.nn.Module):

    def __init__(self, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        # self.encoder = T.nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = T.nn.Linear(d_model,4)

    #     self.init_weights()

    # def init_weights(self) -> None:
    #     initrange = 0.1
    #     # self.encoder.weight.data.uniform_(-initrange, initrange)
    #     self.decoder.bias.data.zero_()
    #     self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: T.Tensor, src_mask: T.Tensor) -> T.Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        # src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> T.Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return T.triu(T.ones(sz, sz) * float('-inf'), diagonal=1)


#in og paper, learned and fixed positional encodings produce similar results
#uses sinusoial (cos and sin) so that any fixed offset is a linear function. also could help model extrapolate to further positions
class PositionalEncoding(T.nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = T.nn.Dropout(p=dropout)

        position = T.arange(max_len).unsqueeze(1)
        div_term = T.exp(T.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)) # four terms of increasingly smaller values
        pe = T.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = T.sin(position * div_term) #every second term
        pe[:, 0, 1::2] = T.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: T.Tensor) -> T.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:

# ntokens = 8  # size of vocabulary
emsize = 8  # embedding dimension, # of features
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(emsize,nhead,d_hid,nlayers,dropout)
#use https://en.wikipedia.org/wiki/Perplexity to evaluate? 
bptt = 30.0
def generate_square_subsequent_mask(sz: int) -> torch.Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
src_mask = generate_square_subsequent_mask(bptt)

criterion = torch.nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
    data, targets = get_batch(train_data, i)
    seq_len = data.size(0)
    if seq_len != bptt:  # only on last batch
        src_mask = src_mask[:seq_len, :seq_len]
    print(data.shape)
    output = model(data, src_mask)
    print(output.shape)
    loss = criterion(output.view(-1, 8), targets)
    # print(targets)
    break

In [ ]:
import torch.nn as nn
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.rand(5, 5, 512)
out = encoder_layer(src)